In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps
import random
from datetime import datetime
# Import the time module.
import time
# Get today's date in seconds.
today = time.time()
# Import the API key.
from config import weather_api_key

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90., high=90., size=2000)
lngs = np.random.uniform(low=-180., high=180., size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [5]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

740

In [6]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=14d496981f3ab4ab4fbc15adae63ab96


In [7]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        print(city_lat)
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        #city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_description
                         })

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | carnarvon
-24.8667
Processing Record 2 of Set 1 | vagur
61.4732
Processing Record 3 of Set 1 | ushuaia
-54.8
Processing Record 4 of Set 1 | bluff
-46.6
Processing Record 5 of Set 1 | cape town
-33.9258
Processing Record 6 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 7 of Set 1 | lithakia
37.7167
Processing Record 8 of Set 1 | okandja
City not found. Skipping...
Processing Record 9 of Set 1 | khonuu
City not found. Skipping...
Processing Record 10 of Set 1 | vila franca do campo
37.7167
Processing Record 11 of Set 1 | ambatofinandrahana
-20.55
Processing Record 12 of Set 1 | fairhope
30.523
Processing Record 13 of Set 1 | iqaluit
63.7506
Processing Record 14 of Set 1 | chateau-thierry
49.05
Processing Record 15 of Set 1 | warqla
City not found. Skipping...
Processing Record 16 of Set 1 | belushya guba
City not found. Skipping...
Processing Record 17 of Set 1 | barentsburg
Cit

-33.0393
Processing Record 11 of Set 4 | pangnirtung
66.1451
Processing Record 12 of Set 4 | avera
33.194
Processing Record 13 of Set 4 | butaritari
3.0707
Processing Record 14 of Set 4 | at-bashi
41.1702
Processing Record 15 of Set 4 | port blair
11.6667
Processing Record 16 of Set 4 | banjar
-8.19
Processing Record 17 of Set 4 | mimongo
-1.6195
Processing Record 18 of Set 4 | grand gaube
-20.0064
Processing Record 19 of Set 4 | faanui
-16.4833
Processing Record 20 of Set 4 | hobyo
5.3505
Processing Record 21 of Set 4 | chokurdakh
70.6333
Processing Record 22 of Set 4 | hambantota
6.1241
Processing Record 23 of Set 4 | preston
53.7667
Processing Record 24 of Set 4 | anadyr
64.75
Processing Record 25 of Set 4 | tasiilaq
65.6145
Processing Record 26 of Set 4 | camacha
33.0833
Processing Record 27 of Set 4 | kupang
-10.1667
Processing Record 28 of Set 4 | dikson
73.5069
Processing Record 29 of Set 4 | san cristobal
7.7669
Processing Record 30 of Set 4 | san quintin
30.4833
Processing Rec

22.8909
Processing Record 21 of Set 7 | saint-pierre
-21.3393
Processing Record 22 of Set 7 | touros
-5.1989
Processing Record 23 of Set 7 | baruun-urt
46.6806
Processing Record 24 of Set 7 | hervey bay
-25.2986
Processing Record 25 of Set 7 | aklavik
68.2191
Processing Record 26 of Set 7 | amderma
City not found. Skipping...
Processing Record 27 of Set 7 | karauzyak
City not found. Skipping...
Processing Record 28 of Set 7 | ust-koksa
50.2696
Processing Record 29 of Set 7 | korla
41.7597
Processing Record 30 of Set 7 | chuy
-33.6971
Processing Record 31 of Set 7 | muhos
64.808
Processing Record 32 of Set 7 | ocampo
21.65
Processing Record 33 of Set 7 | antofagasta
-23.65
Processing Record 34 of Set 7 | isangel
-19.55
Processing Record 35 of Set 7 | kendari
-3.945
Processing Record 36 of Set 7 | bolobo
-2.1667
Processing Record 37 of Set 7 | ternate
0.8
Processing Record 38 of Set 7 | makakilo city
21.3469
Processing Record 39 of Set 7 | nsoko
-27.0333
Processing Record 40 of Set 7 | l

-17.8167
Processing Record 34 of Set 10 | doka
13.5167
Processing Record 35 of Set 10 | besikduzu
41.052
Processing Record 36 of Set 10 | moose factory
51.2585
Processing Record 37 of Set 10 | mvomero
-6.25
Processing Record 38 of Set 10 | dhaka
23.7104
Processing Record 39 of Set 10 | lyubytino
58.8122
Processing Record 40 of Set 10 | itarema
-2.9248
Processing Record 41 of Set 10 | concordia
-31.393
Processing Record 42 of Set 10 | terra santa
-2.1042
Processing Record 43 of Set 10 | bathsheba
13.2167
Processing Record 44 of Set 10 | frontera
26.9333
Processing Record 45 of Set 10 | kondinskoye
59.6481
Processing Record 46 of Set 10 | iquique
-20.2208
Processing Record 47 of Set 10 | rafaela
-31.2503
Processing Record 48 of Set 10 | conway
35.0887
Processing Record 49 of Set 10 | zlotoryja
51.1264
Processing Record 50 of Set 10 | jieshi
22.8134
Processing Record 1 of Set 11 | reporoa
-38.4333
Processing Record 2 of Set 11 | mishan
45.55
Processing Record 3 of Set 11 | toliary
City no

64.5667
Processing Record 45 of Set 13 | zeya
53.75
Processing Record 46 of Set 13 | karpathos
35.507
Processing Record 47 of Set 13 | santiago del estero
-27.7951
Processing Record 48 of Set 13 | atambua
-9.1061
Processing Record 49 of Set 13 | bereda
43.2686
Processing Record 50 of Set 13 | qasigiannguit
68.8193
Processing Record 1 of Set 14 | sambava
-14.2667
Processing Record 2 of Set 14 | daru
-9.0763
Processing Record 3 of Set 14 | channel-port aux basques
47.5702
Processing Record 4 of Set 14 | honningsvag
70.9821
Processing Record 5 of Set 14 | shirokiy
49.758
Processing Record 6 of Set 14 | vostok
46.4856
Processing Record 7 of Set 14 | auki
-8.7676
Processing Record 8 of Set 14 | almaznyy
48.0448
Processing Record 9 of Set 14 | maxixe
-23.8597
Processing Record 10 of Set 14 | kununurra
-15.7667
Processing Record 11 of Set 14 | kaniv
49.7518
Processing Record 12 of Set 14 | mapiri
-15.25
Processing Record 13 of Set 14 | pandan
14.0453
Processing Record 14 of Set 14 | cape eliz

In [10]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)[['City', 'Country', 'Lat','Lng','Max Temp','Humidity','Cloudiness','Wind Speed', 'Current Description']]
city_data_df["Humidity"] = city_data_df["Humidity"].map("{:,.2f}%".format)
city_data_df["Cloudiness"] = city_data_df["Cloudiness"].map("{:,.2f}%".format)
city_data_df["Max Temp"] = city_data_df["Max Temp"]
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Carnarvon,AU,-24.8667,113.6333,62.67,63.00%,75.00%,13.80,broken clouds
1,Vagur,FO,61.4732,-6.8119,48.16,81.00%,100.00%,24.02,overcast clouds
2,Ushuaia,AR,-54.8000,-68.3000,46.02,65.00%,40.00%,26.46,light rain
3,Bluff,NZ,-46.6000,168.3333,52.81,76.00%,24.00%,6.11,few clouds
4,Cape Town,ZA,-33.9258,18.4232,59.09,56.00%,0.00%,4.00,clear sky
5,Lithakia,GR,37.7167,20.8333,68.59,59.00%,0.00%,4.23,clear sky
6,Vila Franca Do Campo,PT,37.7167,-25.4333,75.20,86.00%,100.00%,3.78,overcast clouds
7,Ambatofinandrahana,MG,-20.5500,46.8000,60.96,79.00%,21.00%,2.89,few clouds
8,Fairhope,US,30.5230,-87.9033,82.06,77.00%,40.00%,6.91,scattered clouds
9,Iqaluit,CA,63.7506,-68.5145,37.13,60.00%,75.00%,0.00,broken clouds


In [11]:
city_data_df.to_csv('WeatherPy_Database.csv')